In [1]:
import pandas as pd 
from pathlib import Path
ROOT_DIR = str(Path().resolve()) + '\data'
co_df = pd.read_csv( ROOT_DIR + '\co2\Carbon_dioxide_Gt_CO2_yr.csv')
df = pd.read_csv(ROOT_DIR + '\co2\CO2_global_CEDS_emissions_by_sector_2021_02_05.csv')
df = pd.DataFrame(df.sum(axis = 0)).iloc[3:].reset_index()
df['index'] = df['index'].apply(lambda x: int(x[1:]))
df = df[df['index'] < 2015]
df.columns = ['years', 'historic']
years =  list(df.years) + list(co_df.years)
ssp119 = list(df.historic) + list(co_df.ssp119*10**6)
ssp126 = list(df.historic) + list(co_df.ssp126*10**6)
ssp585 = list(df.historic) + list(co_df.ssp585*10**6)
new_df = pd.DataFrame([years,ssp119,ssp126,ssp585]).T
new_df.columns = ['years', 'ssp119', 'ssp126', 'ssp585']
new_df.to_csv(ROOT_DIR + '\co2\\aggregated.csv', index = False)

In [ ]:
import pandas as pd 
import os 
import json
import dash_leaflet.express as dlx
path = r'D:\Stage\Climat\Giec\new_webapp\giec\data\secheresse'

def point_to_poly(point_geo,scale = 0.5, xscale = 1):
    geometry = point_geo['geometry']
    geometry['type'] = 'Polygon'
    coord = geometry['coordinates']
    list_coord = [[-scale*xscale,-scale],[scale*xscale,-scale],[scale*xscale,scale],[-scale*xscale,scale]]
    list_coord = [ [coord[0] + a , coord[1] + b] for [a,b] in list_coord]
    list_coord.append(list_coord[0])
    geometry['coordinates'] = [list_coord]
    return point_geo

def treat_secheresse(path):
    dict_df = {}
    for file in os.scandir(path):
        df = pd.read_csv(file.path,comment = '#', encoding= 'ISO-8859-1', delimiter = ';').drop(columns= ['Point', 'Contexte', 'Unnamed: 6'])
        horizons = list(df.iloc[:,2].drop_duplicates())
        print(horizons)
        for horizon in horizons:
            nom = file.name[:-4] + horizon
            code = 'NORSPI' 
            temp_df = df[df.iloc[:,2] == horizon]
            temp_df.columns = ['tooltip' if x==code else x for x in df.columns]
            dico = temp_df.to_dict('rows')
            geo_json = dlx.dicts_to_geojson(dico, lon = 'Longitude', lat = 'Latitude')

            for element in geo_json['features'] : 
                element = point_to_poly(element, scale = 0.0357,xscale = 1.44)
            dict_df.update({nom:geo_json})
            with open(path + '/' + nom+  '.geojson', 'w') as f:
                json.dump(geo_json, f)
    return dict_df
treat_secheresse(path)

In [5]:
import plotly.express as px
def rgb_to_hex(rgb):
    return '#'+ '%02x%02x%02x' % rgb
rgb  =[
     (161, 105, 40),
     (189, 146, 90),
     (214, 189, 141),
  (237, 234, 194),
  (181, 200, 184),
  (121, 167, 172),
  (40, 135, 161)]
[rgb_to_hex(col) for col in rgb]
import numpy as np


['#a16928', '#bd925a', '#d6bd8d', '#edeac2', '#b5c8b8', '#79a7ac', '#2887a1']